Peut être lent au départ car l'hébergement est sur Heroku en "low cost" à mon avis

The API is limited to 5 calls/minutes. 

If you try more, your endpoint will throw back an error.

Les features du CSV (https://lead-program-assets.s3.eu-west-3.amazonaws.com/M05-Projects/fraudTest.csv)
* ,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
* 0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,29209,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
* 1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.436,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450497999999996,-109.960431,0

- vide;
- trans_date_trans_time;  # ! N APPARAIT PAS DANS LES DONNEES FOURNIES PAR L'API
- cc_num;
- merchant;
- category;
- amt;
- first;
- last;
- gender;
- street;
- city;
- state;
- zip;
- lat;
- long;
- city_pop;
- job;
- dob;
- trans_num;
- unix_time; # ! APPARAIT EN PLUS DANS LES DONNEES FOURNIES PAR L'API
- merch_lat;
- merch_long;
- is_fraud


In [ ]:

import json
import requests
import pandas as pd
from datetime import datetime, timezone

pd.set_option('display.max_columns', None)

url = "https://real-time-payments-api.herokuapp.com/current-transactions"

response = requests.get(url)
data = response.json()

print("\n1ere lecture des données")
print(data)

if isinstance(data, str):
    data = json.loads(data)
    print("\n2nde lecture des données")
    print(data)


columns = data['columns']
index = data['index']
rows = data['data']

df = pd.DataFrame(data=rows, index=index, columns=columns)


# ! 17 is hard coded 
col = df['current_time']
df.insert(17, "unix_time", col)

# convert to date string
df.rename(columns={'current_time': 'trans_date_trans_time'}, inplace=True)
timestamp = df["trans_date_trans_time"].iloc[0]
date = datetime.fromtimestamp(timestamp / 1000, tz=timezone.utc)
str_date = date.strftime('%Y-%m-%d %H:%M:%S')

df['trans_date_trans_time'] = df['trans_date_trans_time'].astype(str)
df.at[index[0], "trans_date_trans_time"] = str_date

# Modifies the order of columns in the df DataFrame
# Moving the last column to the first position
# Leaving all other columns in their original order
cols = df.columns.tolist()
reordered_cols = [cols[-1]] + cols[:-1]  # the last col then all the other until the before last col
df = df[reordered_cols]


print("\nDisplay du dataframe")
display(df)

# Convert DataFrame back to JSON with original format
bob = {
    "columns": df.columns.tolist(),
    "index": df.index.tolist(),
    "data": df.values.tolist()
}

# Convert to JSON string
# bob_json = json.dumps(bob, indent=4)

print("\nAffichage du dataframe en format json")
print(bob)


1ere lecture des données
{"columns":["cc_num","merchant","category","amt","first","last","gender","street","city","state","zip","lat","long","city_pop","job","dob","trans_num","merch_lat","merch_long","is_fraud","current_time"],"index":[349827],"data":[[4066595222529,"fraud_Prohaska-Murray","gas_transport",44.55,"Jamie","Olson","F","3423 Michael Canyon Suite 276","River","KY",41254,37.8795,-82.7251,571,"Quarry manager","1931-03-07","c1401442664895668be9de1005931536",37.580406,-82.199441,0,1730158296851]]}

2nde lecture des données
{'columns': ['cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'merch_lat', 'merch_long', 'is_fraud', 'current_time'], 'index': [349827], 'data': [[4066595222529, 'fraud_Prohaska-Murray', 'gas_transport', 44.55, 'Jamie', 'Olson', 'F', '3423 Michael Canyon Suite 276', 'River', 'KY', 41254, 37.8795, -82.7251, 571, 'Quarry manager', '1931-03-07', 'c14014426

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
349827,2024-10-28 23:31:36,4066595222529,fraud_Prohaska-Murray,gas_transport,44.55,Jamie,Olson,F,3423 Michael Canyon Suite 276,River,KY,41254,37.8795,-82.7251,571,Quarry manager,1931-03-07,c1401442664895668be9de1005931536,1730158296851,37.580406,-82.199441,0



Affichage du dataframe en format json
{'columns': ['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud'], 'index': [349827], 'data': [['2024-10-28 23:31:36', 4066595222529, 'fraud_Prohaska-Murray', 'gas_transport', 44.55, 'Jamie', 'Olson', 'F', '3423 Michael Canyon Suite 276', 'River', 'KY', 41254, 37.8795, -82.7251, 571, 'Quarry manager', '1931-03-07', 'c1401442664895668be9de1005931536', 1730158296851, 37.580406, -82.199441, 0]]}
